In [3]:
import torch
import os
data = torch.load(os.path.join("data", "datasets", "SlateTopKBoredInfv0numitem100slatesize10_oracle_epsilon1.0_seed2023_n_users10.pt"))
item_embeddings = torch.load(os.path.join("data", "datasets", "embeddings", "SlateTopKBoredInfv0numitem100slatesize10_oracle_epsilon1.0_seed2023_n_users10.pt"))


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\Yassi\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Yassi\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\Yassi\Documents\GitHub\Master_thesis_RLRSs\sardine\embeddings\semi_synthetic\datasets-venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\Yassi\Documents\GitHub\M

In [4]:
data.observations.keys()

dict_keys(['clicks', 'hist', 'slate'])

In [5]:
data.observations["slate"]

clicked_items_per_user = []

user_list = []
clicked_items = []

for i in range(len(data.observations["slate"])):
    slate =data.observations["slate"][i][0]
    clicks = data.observations["clicks"][i][0]
    # if the item was clicked, add it to the list
    for j in range(len(slate)):
        if clicks[j] == 1:
            clicked_items.append(int(slate[j]))
    if (i + 1) % 100 == 0:
        clicked_items_per_user.append(clicked_items)
        clicked_items = [] 


# create a user matrix where the rows are the users and the columns are the items. The clicked items are marked as the number of times a user has clicked on them
user_matrix = torch.zeros(len(clicked_items_per_user), 100)
for i in range(len(clicked_items_per_user)):
    for j in range(len(clicked_items_per_user[i])):
        user_matrix[i][clicked_items_per_user[i][j]] += 1
        
user_matrix

tensor([[ 0.,  0.,  1.,  0.,  0.,  6.,  0.,  0.,  0.,  0.,  0.,  6.,  3.,  0.,
          0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  5.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.

In [6]:
# create matrix factorization model based on the user matrix and the item embeddings with an embedding dimension of 10
import torch
import torch.nn as nn
import torch.optim as optim

class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_items, embedding_dim):
        super(MatrixFactorization, self).__init__()
        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        self.item_embedding = nn.Embedding(n_items, embedding_dim)
        
    def forward(self, user, item):
        user_embedding = self.user_embedding(user)
        item_embedding = self.item_embedding(item)
        return (user_embedding * item_embedding).sum(1)

n_users = user_matrix.size(0)

n_items = item_embeddings.size(0)

embedding_dim = 10

model = MatrixFactorization(n_users, n_items, embedding_dim)

# define the loss function and the optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# train the model
for epoch in range(100):
    optimizer.zero_grad()
    user = torch.LongTensor(range(n_users))
    item = torch.LongTensor(range(n_items))
    prediction = model(user, item)
    loss = criterion(prediction, user_matrix.view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# get the user embeddings
user_embeddings = model.user_embedding.weight.data

user_embeddings

TypeError: 'int' object is not callable